# Memory Management
(cont:mm:outline)=
Outline of memory management:
- Fundamental jobs of the memory manager:
    - Abstracting a useful memory model.
    - Managing the physical memory on the system.
    - Providing and managing a private virtual address space for each user.
- The evolution of memory management:
    - Simple physical memory management.
        - Sharing one physical address space.
        - Relocating programs.
        - Swapping whole programs.
        - Compaction.
    - Virtual memory management.
        - Segmentation.
            - Single segments per address space
            - Multiple segments per address space.
            - Swapping.
            - Compaction.
        - Paging.
            - Virtual pages and page frames.
            - Virtual address spaces larger than RAM.
            - Page tables.
- Modern memory management systems
    - Demand Paging
        - Several very large sparse virtual address spaces.
        - Millions or Billions of physical memory page frames.
    - Managing the pages of physical memory.
        - Page structures
        - Page lists
           - Free lists
               - coalescing free memory.
           - In-use page lists
               - active and inactive lists for reclaiming.
               - Over-committing physical memory
       - Over-committing physical memory
           - Reclaiming memory
           - paging and swapping
       - Reclaiming memory
           - paging and swapping
   - Maintaining a separate virtual address space for each process.
       - Types of virtual memory regions:
           - Mapped file memory regions
           - Anonymous memory regions
       - Page tables
           - Multiple level page tables
           - Page table entries
           - Inverted page tables
           - Variable page sizes
       - Page Faults
           - File backed page faults
           - Anonymous page faults
           - Copy On Write page faults
   - Memory reclaiming algorithms.
       - Global page reclaiming
           - FIFO, LRU, NRU
       - Local page reclaiming
           - Reclaiming within a specific address space
       - Working set page reclaiming
   - Buffer Cache
       - Using RAM to cache file system data & metadata
       - Global page hash
       - Per file page hash
   - Paging Dynamics
       - Boot-time
       - Consuming physical memory
           - Kernel page wiring
           - Page faults
           - Buffer Cache
       - Freeing physical memory
           - Process exiting
           - un-mapping
           - Page reclaiming
               - Paging
               - Swapping
               - Page cache purging
       - System shutdown
   - Memory Management Hardware
       - Translations
           - Page Table entries
       - TLB - Transpation Lookaside Buffers and entries
           - Global TLB entries
           - Local TLB entries
           - Tagging TLB entries
   - Linux specific memory management
 
We first talk about the [tasks](cont:mm:task) of the operating system memory management. 

- why do we need memory mangement sharing memory between different processes, what is ram...
- physical memory management
- virtual memory management
  - segmentation
  - paging


(cont:mm:task)=
## Task of Memory Management
The memory manager is responsible for 1.) Managing all of the RAM or physical memory on the whole system, 2.) Creating and managing virtual memory for each process and 3.) transitioning the processor(s) between physical memory mode and virtual memory mode as the kernel runs.

(cont:mm:physem)=
## Physical Memory
Physical memory can be viewed as a simple array of bytes.
 

(cont:mm:virtmem)=
## Virtual Memory
Virtual memory is the address space that physical memory is mapped.

(cont:mm:modes)=
## Memory Modes
Physical memory mode implies that no translation occurs.
Virtual memory mode implies that virtual memory addresses are translated to to physical memory addresses.

## Notes

Refer [here](https://jappavoo.github.io/UndertheCovers/textbook/unix/shellintro.html#arithmetic-expansion) for doing binary math in shell 